In [1]:
%load_ext lab_black

In [17]:
import sqlite3
from pathlib import Path

import pandas as pd

In [3]:
data_dir = Path("./data")
assert data_dir.exists()
gene_node_file = data_dir / "gene_node.csv"
assert gene_node_file.exists()
gwas_node_file = data_dir / "gwas_node.csv"
assert gwas_node_file.exists()
edge_file = data_dir / "mrres.csv"
assert edge_file.exists()

output_dir = data_dir / "output"
output_dir.mkdir(exist_ok=True)

# Init

In [19]:
STUDY_SOURCE = "trans-ancestry-pwas_2021-11-01"

## gene nodes

In [4]:
gene_node_raw = pd.read_csv(gene_node_file)
print(gene_node_raw.info())
gene_node_raw

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1477 entries, 0 to 1476
Data columns (total 4 columns):
_id            1477 non-null object
_name          1477 non-null object
chr            1477 non-null int64
description    1477 non-null object
dtypes: int64(1), object(3)
memory usage: 46.3+ KB
None


,_id,_name,chr,description
0,ENSG00000168685,IL7R,5,P16871;Interleukin-7 receptor subunit alpha
1,ENSG00000146701,MDH2,7,P40926;Malate dehydrogenase/ mitochondrial
2,ENSG00000023892,DEF6,6,Q9H4E7;Differentially expressed in FDCP 6 homolog
3,ENSG00000109854,HTATIP2,11,Q9BUP3;Oxidoreductase HTATIP2
4,ENSG00000174080,CTSF,11,Q9UBX1;Cathepsin F
...,...,...,...,...
1472,ENSG00000108602,ALDH3A1,17,P30838;Aldehyde dehydrogenase/ dimeric NADP-pr...
1473,ENSG00000164144,ARFIP1,4,P53367;Arfaptin-1
1474,ENSG00000120885,CLU,8,P10909;Clusterin
1475,ENSG00000163736,PPBP,4,P02775;Platelet basic protein


In [5]:
gene_node = gene_node_raw.dropna()
print(gene_node.info())
gene_node

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1477 entries, 0 to 1476
Data columns (total 4 columns):
_id            1477 non-null object
_name          1477 non-null object
chr            1477 non-null int64
description    1477 non-null object
dtypes: int64(1), object(3)
memory usage: 57.7+ KB
None


,_id,_name,chr,description
0,ENSG00000168685,IL7R,5,P16871;Interleukin-7 receptor subunit alpha
1,ENSG00000146701,MDH2,7,P40926;Malate dehydrogenase/ mitochondrial
2,ENSG00000023892,DEF6,6,Q9H4E7;Differentially expressed in FDCP 6 homolog
3,ENSG00000109854,HTATIP2,11,Q9BUP3;Oxidoreductase HTATIP2
4,ENSG00000174080,CTSF,11,Q9UBX1;Cathepsin F
...,...,...,...,...
1472,ENSG00000108602,ALDH3A1,17,P30838;Aldehyde dehydrogenase/ dimeric NADP-pr...
1473,ENSG00000164144,ARFIP1,4,P53367;Arfaptin-1
1474,ENSG00000120885,CLU,8,P10909;Clusterin
1475,ENSG00000163736,PPBP,4,P02775;Platelet basic protein


In [6]:
print(gene_node["_id"].drop_duplicates().pipe(len))
print(gene_node["_name"].drop_duplicates().pipe(len))

1477
1477


In [7]:
dup_genes = (
    gene_node[["_id", "_name"]]
    .groupby("_id")
    .size()
    .reset_index(name="counts")
    .query("counts > 1")
)
print(dup_genes.head())
dup_df = gene_node[gene_node["_id"].isin(dup_genes["_id"])]
dup_df.sort_values("_id")

Empty DataFrame
Columns: [_id, counts]
Index: []


,_id,_name,chr,description


## gwas nodes

In [8]:
gwas_node_raw = pd.read_csv(gwas_node_file)
print(gwas_node_raw.info())
gwas_node_raw

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 16 columns):
_id            16 non-null object
_name          16 non-null object
consortium     16 non-null object
build          16 non-null object
ncase          16 non-null int64
ncontrol       16 non-null int64
sample_size    16 non-null int64
nsnp           16 non-null int64
population     16 non-null object
sex            16 non-null object
author         16 non-null object
year           16 non-null int64
unit           16 non-null object
mr             16 non-null int64
category       16 non-null object
trait          16 non-null object
dtypes: int64(6), object(10)
memory usage: 2.1+ KB
None


,_id,_name,consortium,build,ncase,ncontrol,sample_size,nsnp,population,sex,author,year,unit,mr,category,trait
0,gbmi-a-00001-afr-b,Asthma,GBMI (Global biobank Meta-analysis initiative),HG38/GRCh38,5054,27599,32653,24660533,African,Males/Females,Wei Zhou,2021,log odds ratio,1,Binary,Asthma
1,gbmi-a-00001-nfe-b,Asthma,GBMI (Global biobank Meta-analysis initiative),HG38/GRCh38,100736,1118479,1219215,47156680,European,Males/Females,Wei Zhou,2021,log odds ratio,1,Binary,Asthma
2,gbmi-a-00002-afr-b,Chronic obstructive pulmonary disease (COPD),GBMI (Global biobank Meta-analysis initiative),HG38/GRCh38,1978,27699,29677,23889093,African,Males/Females,Wei Zhou,2021,log odds ratio,1,Binary,Chronic obstructive pulmonary disease (COPD)
3,gbmi-a-00002-nfe-b,Chronic obstructive pulmonary disease (COPD),GBMI (Global biobank Meta-analysis initiative),HG38/GRCh38,51231,749346,800577,45532064,European,Males/Females,Wei Zhou,2021,log odds ratio,1,Binary,Chronic obstructive pulmonary disease (COPD)
4,gbmi-a-00003-afr-b,Gout,GBMI (Global biobank Meta-analysis initiative),HG38/GRCh38,1313,33896,35209,25056505,African,Males/Females,Wei Zhou,2021,log odds ratio,1,Binary,Gout
5,gbmi-a-00003-nfe-b,Gout,GBMI (Global biobank Meta-analysis initiative),HG38/GRCh38,20512,822995,843507,47218578,European,Males/Females,Wei Zhou,2021,log odds ratio,1,Binary,Gout
6,gbmi-a-00004-afr-b,Heart Failure (HF),GBMI (Global biobank Meta-analysis initiative),HG38/GRCh38,970,16823,17793,24631161,African,Males/Females,Wei Zhou,2021,log odds ratio,1,Binary,Heart Failure (HF)
7,gbmi-a-00004-nfe-b,Heart Failure (HF),GBMI (Global biobank Meta-analysis initiative),HG38/GRCh38,27064,769538,796602,46480843,European,Males/Females,Wei Zhou,2021,log odds ratio,1,Binary,Heart Failure (HF)
8,gbmi-a-00005-afr-b,Idiopathic pulmonary fibrosis (IPF),GBMI (Global biobank Meta-analysis initiative),HG38/GRCh38,168,8364,8532,18058634,African,Males/Females,Wei Zhou,2021,log odds ratio,1,Binary,Idiopathic pulmonary fibrosis (IPF)
9,gbmi-a-00005-nfe-b,Idiopathic pulmonary fibrosis (IPF),GBMI (Global biobank Meta-analysis initiative),HG38/GRCh38,4066,747896,751962,45621626,European,Males/Females,Wei Zhou,2021,log odds ratio,1,Binary,Idiopathic pulmonary fibrosis (IPF)


In [9]:
print(gwas_node_raw["_id"].drop_duplicates().pipe(len))
print(gwas_node_raw["_name"].drop_duplicates().pipe(len))

16
8


## edge

In [10]:
edge_raw = pd.read_csv(edge_file)
print(edge_raw.info())
edge_raw

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19582 entries, 0 to 19581
Data columns (total 20 columns):
_id            19582 non-null object
_name          19582 non-null object
chr            19582 non-null int64
protein        19582 non-null object
description    19582 non-null object
ancestry       19582 non-null object
disease        19582 non-null object
method         19582 non-null object
nsnp           19582 non-null int64
b              19582 non-null float64
ci_low         14945 non-null float64
ci_upp         14945 non-null float64
se             19582 non-null float64
pval           19582 non-null float64
selection      19582 non-null object
moescore       19582 non-null int64
ldcheck        1224 non-null object
pwcoco         1224 non-null object
pleiotropy     9656 non-null object
seqid          19582 non-null object
dtypes: float64(5), int64(3), object(12)
memory usage: 3.0+ MB
None


,_id,_name,chr,protein,description,ancestry,disease,method,nsnp,b,ci_low,ci_upp,se,pval,selection,moescore,ldcheck,pwcoco,pleiotropy,seqid
0,ENSG00000168685,IL7R,5,IL-7 Ra,P16871;Interleukin-7 receptor subunit alpha,European,gbmi-a-00001-nfe-b,Wald_ratio,1,0.135224,NaN,NaN,0.046594,3.705908e-03,DF+HF,1,True,False,NaN,SeqId_5089_11
1,ENSG00000146701,MDH2,7,Malate dehydrogenase 2,P40926;Malate dehydrogenase/ mitochondrial,European,gbmi-a-00001-nfe-b,Wald_ratio,1,0.066043,NaN,NaN,0.034404,5.490592e-02,DF+HF,1,NaN,NaN,NaN,SeqId_15534_26
2,ENSG00000023892,DEF6,6,DEFI6,Q9H4E7;Differentially expressed in FDCP 6 homolog,European,gbmi-a-00001-nfe-b,Wald_ratio,1,-0.256950,NaN,NaN,0.051617,6.430000e-07,DF+HF,1,True,True,NaN,SeqId_14090_23
3,ENSG00000109854,HTATIP2,11,HTAI2,Q9BUP3;Oxidoreductase HTATIP2,European,gbmi-a-00001-nfe-b,Wald_ratio,1,-0.035607,NaN,NaN,0.017421,4.096695e-02,DF+HF,1,False,False,NaN,SeqId_10630_5
4,ENSG00000174080,CTSF,11,CATF,Q9UBX1;Cathepsin F,European,gbmi-a-00001-nfe-b,Wald_ratio,1,-0.014405,NaN,NaN,0.020015,4.717140e-01,DF+HF,1,NaN,NaN,NaN,SeqId_9212_22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19577,ENSG00000120256,LRP11,6,LRP11,Q86VZ4;Low-density lipoprotein receptor-relate...,African,gbmi-a-00004-afr-b,generalized IVW,8,0.123291,-0.066526,0.038228,0.048226,1.057282e-02,DF+HF,1,False,False,False,SeqId_15472_16
19578,ENSG00000105376,ICAM5,19,sICAM-5,Q9UMF0;Intercellular adhesion molecule 5,African,gbmi-a-00004-afr-b,generalized IVW,2,-0.085452,-0.062592,0.109604,0.078178,2.743783e-01,DF+HF,1,NaN,NaN,NaN,SeqId_8245_27
19579,ENSG00000028137,TNFRSF1B,1,TNF sR-II,P20333;Tumor necrosis factor receptor superfam...,African,gbmi-a-00004-afr-b,generalized IVW,3,0.047461,NaN,NaN,0.095236,6.182397e-01,DF+HF,1,NaN,NaN,False,SeqId_8368_102
19580,ENSG00000274736,CCL23,17,C-C motif chemokine 23,P55773;C-C motif chemokine 23,African,gbmi-a-00004-afr-b,generalized IVW,3,0.110918,-0.089456,0.067650,0.084080,1.871027e-01,DF+HF,1,NaN,NaN,False,SeqId_2913_1


In [11]:
dup_edge_check = (
    edge_raw.groupby(["_id", "disease"])
    .size()
    .reset_index(name="counts")
    .query("counts > 1")
)
dup_edge_check

,_id,disease,counts


In [12]:
dup_gene_ids = dup_df["_id"].drop_duplicates()
foobar = edge_raw[
    (edge_raw["_id"].isin(dup_gene_ids)) & (edge_raw["disease"] == "gbmi-a-00001-nfe-b")
]
print(foobar.info())
print(foobar["_id"].drop_duplicates().pipe(len))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 20 columns):
_id            0 non-null object
_name          0 non-null object
chr            0 non-null int64
protein        0 non-null object
description    0 non-null object
ancestry       0 non-null object
disease        0 non-null object
method         0 non-null object
nsnp           0 non-null int64
b              0 non-null float64
ci_low         0 non-null float64
ci_upp         0 non-null float64
se             0 non-null float64
pval           0 non-null float64
selection      0 non-null object
moescore       0 non-null int64
ldcheck        0 non-null object
pwcoco         0 non-null object
pleiotropy     0 non-null object
seqid          0 non-null object
dtypes: float64(5), int64(3), object(12)
memory usage: 0.0+ bytes
None
0


# Cleaning

In [29]:
xqtl_pwas_mr = edge_raw.rename(
    columns={"_id": "gene_id", "_name": "gene_name", "disease": "gwas_id"}
).merge(
    gwas_node_raw[["_id", "_name"]]
    .rename(columns={"_id": "gwas_id", "_name": "gwas_name"})
    .drop_duplicates(),
    left_on="gwas_id",
    right_on="gwas_id",
)
print(xqtl_pwas_mr.info())
xqtl_pwas_mr

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19582 entries, 0 to 19581
Data columns (total 21 columns):
gene_id        19582 non-null object
gene_name      19582 non-null object
chr            19582 non-null int64
protein        19582 non-null object
description    19582 non-null object
ancestry       19582 non-null object
gwas_id        19582 non-null object
method         19582 non-null object
nsnp           19582 non-null int64
b              19582 non-null float64
ci_low         14945 non-null float64
ci_upp         14945 non-null float64
se             19582 non-null float64
pval           19582 non-null float64
selection      19582 non-null object
moescore       19582 non-null int64
ldcheck        1224 non-null object
pwcoco         1224 non-null object
pleiotropy     9656 non-null object
seqid          19582 non-null object
gwas_name      19582 non-null object
dtypes: float64(5), int64(3), object(13)
memory usage: 3.3+ MB
None


,gene_id,gene_name,chr,protein,description,ancestry,gwas_id,method,nsnp,b,...,ci_upp,se,pval,selection,moescore,ldcheck,pwcoco,pleiotropy,seqid,gwas_name
0,ENSG00000168685,IL7R,5,IL-7 Ra,P16871;Interleukin-7 receptor subunit alpha,European,gbmi-a-00001-nfe-b,Wald_ratio,1,0.135224,...,NaN,0.046594,3.705908e-03,DF+HF,1,True,False,NaN,SeqId_5089_11,Asthma
1,ENSG00000146701,MDH2,7,Malate dehydrogenase 2,P40926;Malate dehydrogenase/ mitochondrial,European,gbmi-a-00001-nfe-b,Wald_ratio,1,0.066043,...,NaN,0.034404,5.490592e-02,DF+HF,1,NaN,NaN,NaN,SeqId_15534_26,Asthma
2,ENSG00000023892,DEF6,6,DEFI6,Q9H4E7;Differentially expressed in FDCP 6 homolog,European,gbmi-a-00001-nfe-b,Wald_ratio,1,-0.256950,...,NaN,0.051617,6.430000e-07,DF+HF,1,True,True,NaN,SeqId_14090_23,Asthma
3,ENSG00000109854,HTATIP2,11,HTAI2,Q9BUP3;Oxidoreductase HTATIP2,European,gbmi-a-00001-nfe-b,Wald_ratio,1,-0.035607,...,NaN,0.017421,4.096695e-02,DF+HF,1,False,False,NaN,SeqId_10630_5,Asthma
4,ENSG00000174080,CTSF,11,CATF,Q9UBX1;Cathepsin F,European,gbmi-a-00001-nfe-b,Wald_ratio,1,-0.014405,...,NaN,0.020015,4.717140e-01,DF+HF,1,NaN,NaN,NaN,SeqId_9212_22,Asthma
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19577,ENSG00000120256,LRP11,6,LRP11,Q86VZ4;Low-density lipoprotein receptor-relate...,African,gbmi-a-00004-afr-b,generalized IVW,8,0.123291,...,0.038228,0.048226,1.057282e-02,DF+HF,1,False,False,False,SeqId_15472_16,Heart Failure (HF)
19578,ENSG00000105376,ICAM5,19,sICAM-5,Q9UMF0;Intercellular adhesion molecule 5,African,gbmi-a-00004-afr-b,generalized IVW,2,-0.085452,...,0.109604,0.078178,2.743783e-01,DF+HF,1,NaN,NaN,NaN,SeqId_8245_27,Heart Failure (HF)
19579,ENSG00000028137,TNFRSF1B,1,TNF sR-II,P20333;Tumor necrosis factor receptor superfam...,African,gbmi-a-00004-afr-b,generalized IVW,3,0.047461,...,NaN,0.095236,6.182397e-01,DF+HF,1,NaN,NaN,False,SeqId_8368_102,Heart Failure (HF)
19580,ENSG00000274736,CCL23,17,C-C motif chemokine 23,P55773;C-C motif chemokine 23,African,gbmi-a-00004-afr-b,generalized IVW,3,0.110918,...,0.067650,0.084080,1.871027e-01,DF+HF,1,NaN,NaN,False,SeqId_2913_1,Heart Failure (HF)


In [28]:
gwas_nodes = gwas_node_raw[
    gwas_node_raw["_id"].isin(xqtl_pwas_mr["gwas_id"].drop_duplicates())
]
print(gwas_nodes.info())
gwas_nodes

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16 entries, 0 to 15
Data columns (total 16 columns):
_id            16 non-null object
_name          16 non-null object
consortium     16 non-null object
build          16 non-null object
ncase          16 non-null int64
ncontrol       16 non-null int64
sample_size    16 non-null int64
nsnp           16 non-null int64
population     16 non-null object
sex            16 non-null object
author         16 non-null object
year           16 non-null int64
unit           16 non-null object
mr             16 non-null int64
category       16 non-null object
trait          16 non-null object
dtypes: int64(6), object(10)
memory usage: 2.1+ KB
None


,_id,_name,consortium,build,ncase,ncontrol,sample_size,nsnp,population,sex,author,year,unit,mr,category,trait
0,gbmi-a-00001-afr-b,Asthma,GBMI (Global biobank Meta-analysis initiative),HG38/GRCh38,5054,27599,32653,24660533,African,Males/Females,Wei Zhou,2021,log odds ratio,1,Binary,Asthma
1,gbmi-a-00001-nfe-b,Asthma,GBMI (Global biobank Meta-analysis initiative),HG38/GRCh38,100736,1118479,1219215,47156680,European,Males/Females,Wei Zhou,2021,log odds ratio,1,Binary,Asthma
2,gbmi-a-00002-afr-b,Chronic obstructive pulmonary disease (COPD),GBMI (Global biobank Meta-analysis initiative),HG38/GRCh38,1978,27699,29677,23889093,African,Males/Females,Wei Zhou,2021,log odds ratio,1,Binary,Chronic obstructive pulmonary disease (COPD)
3,gbmi-a-00002-nfe-b,Chronic obstructive pulmonary disease (COPD),GBMI (Global biobank Meta-analysis initiative),HG38/GRCh38,51231,749346,800577,45532064,European,Males/Females,Wei Zhou,2021,log odds ratio,1,Binary,Chronic obstructive pulmonary disease (COPD)
4,gbmi-a-00003-afr-b,Gout,GBMI (Global biobank Meta-analysis initiative),HG38/GRCh38,1313,33896,35209,25056505,African,Males/Females,Wei Zhou,2021,log odds ratio,1,Binary,Gout
5,gbmi-a-00003-nfe-b,Gout,GBMI (Global biobank Meta-analysis initiative),HG38/GRCh38,20512,822995,843507,47218578,European,Males/Females,Wei Zhou,2021,log odds ratio,1,Binary,Gout
6,gbmi-a-00004-afr-b,Heart Failure (HF),GBMI (Global biobank Meta-analysis initiative),HG38/GRCh38,970,16823,17793,24631161,African,Males/Females,Wei Zhou,2021,log odds ratio,1,Binary,Heart Failure (HF)
7,gbmi-a-00004-nfe-b,Heart Failure (HF),GBMI (Global biobank Meta-analysis initiative),HG38/GRCh38,27064,769538,796602,46480843,European,Males/Females,Wei Zhou,2021,log odds ratio,1,Binary,Heart Failure (HF)
8,gbmi-a-00005-afr-b,Idiopathic pulmonary fibrosis (IPF),GBMI (Global biobank Meta-analysis initiative),HG38/GRCh38,168,8364,8532,18058634,African,Males/Females,Wei Zhou,2021,log odds ratio,1,Binary,Idiopathic pulmonary fibrosis (IPF)
9,gbmi-a-00005-nfe-b,Idiopathic pulmonary fibrosis (IPF),GBMI (Global biobank Meta-analysis initiative),HG38/GRCh38,4066,747896,751962,45621626,European,Males/Females,Wei Zhou,2021,log odds ratio,1,Binary,Idiopathic pulmonary fibrosis (IPF)


In [27]:
gene_nodes = gene_node_raw.merge(
    xqtl_pwas_mr[["gene_id", "gene_name", "description"]]
    .drop_duplicates()
    .rename(columns={"gene_id": "_id", "gene_name": "_name"}),
    left_on=["_id", "_name", "description"],
    right_on=["_id", "_name", "description"],
)
print(gene_nodes.info())
gene_nodes

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1477 entries, 0 to 1476
Data columns (total 4 columns):
_id            1477 non-null object
_name          1477 non-null object
chr            1477 non-null int64
description    1477 non-null object
dtypes: int64(1), object(3)
memory usage: 57.7+ KB
None


,_id,_name,chr,description
0,ENSG00000168685,IL7R,5,P16871;Interleukin-7 receptor subunit alpha
1,ENSG00000146701,MDH2,7,P40926;Malate dehydrogenase/ mitochondrial
2,ENSG00000023892,DEF6,6,Q9H4E7;Differentially expressed in FDCP 6 homolog
3,ENSG00000109854,HTATIP2,11,Q9BUP3;Oxidoreductase HTATIP2
4,ENSG00000174080,CTSF,11,Q9UBX1;Cathepsin F
...,...,...,...,...
1472,ENSG00000108602,ALDH3A1,17,P30838;Aldehyde dehydrogenase/ dimeric NADP-pr...
1473,ENSG00000164144,ARFIP1,4,P53367;Arfaptin-1
1474,ENSG00000120885,CLU,8,P10909;Clusterin
1475,ENSG00000163736,PPBP,4,P02775;Platelet basic protein


# Post-cleaning validation

In [16]:
# id combination is unique
assert xqtl_pwas_mr.pipe(len) == xqtl_pwas_mr[
    ["gene_id", "gwas_id"]
].drop_duplicates().pipe(len)

# Save

In [34]:
db_path = output_dir / "xqtl_pwas_mr.db"

with sqlite3.connect(db_path) as conn:
    xqtl_pwas_mr.to_sql(
        "XQTL_PWAS_MR", conn, index=True, index_label="idx", if_exists="replace"
    )
    gene_nodes.to_sql("Gene", conn, index=True, index_label="idx", if_exists="replace")
    gwas_nodes.to_sql("Gwas", conn, index=True, index_label="idx", if_exists="replace")

## eval

In [37]:
query = """
SELECT 
    gene_id, gene_name, chr, protein, description,
    gwas_id, gwas_name, ancestry, nsnp,
    b, se, pval, ci_low, ci_upp,
    method, selection, moescore, ldcheck, pwcoco, pleiotropy,
    seqid
FROM
    XQTL_PWAS_MR
WHERE
    gwas_name = "Asthma" AND
    pval <= 0.00001
"""

with sqlite3.connect(db_path) as conn:
    df = pd.read_sql(query, conn)
df

,gene_id,gene_name,chr,protein,description,gwas_id,gwas_name,ancestry,nsnp,b,...,pval,ci_low,ci_upp,method,selection,moescore,ldcheck,pwcoco,pleiotropy,seqid
0,ENSG00000023892,DEF6,6,DEFI6,Q9H4E7;Differentially expressed in FDCP 6 homolog,gbmi-a-00001-nfe-b,Asthma,European,1,-0.256950,...,6.430000e-07,NaN,NaN,Wald_ratio,DF+HF,1,1,1,NaN,SeqId_14090_23
1,ENSG00000204305,AGER,6,sRAGE,Q15109;Advanced glycosylation end product-spec...,gbmi-a-00001-nfe-b,Asthma,European,1,-0.140869,...,1.550000e-18,NaN,NaN,Wald_ratio,DF+HF,1,0,0,NaN,SeqId_4125_52
2,ENSG00000175189,INHBC,12,INHBC,P55103;Inhibin beta C chain,gbmi-a-00001-nfe-b,Asthma,European,1,-0.044609,...,2.970000e-07,NaN,NaN,Wald_ratio,DF+HF,1,1,0,NaN,SeqId_15686_49
3,ENSG00000137507,LRRC32,11,LRC32,Q14392;Leucine-rich repeat-containing protein 32,gbmi-a-00001-nfe-b,Asthma,European,2,0.213974,...,1.810000e-07,0.133607,0.294342,generalized IVW,DF,1,1,0,NaN,SeqId_7551_33
4,ENSG00000174125,TLR1,4,TLR1,Q15399;Toll-like receptor 1,gbmi-a-00001-nfe-b,Asthma,European,2,-0.132174,...,8.040000e-12,-0.170060,-0.094289,generalized IVW,DF,1,1,1,NaN,SeqId_16324_38
5,ENSG00000239697,TNFSF12,17,TWEAK,O43508;Tumor necrosis factor ligand superfamil...,gbmi-a-00001-nfe-b,Asthma,European,2,-0.141647,...,1.500000e-07,-0.194497,-0.088798,generalized IVW,DF+HF,1,1,0,NaN,SeqId_5939_42
6,ENSG00000160856,FCRL3,1,FCRL3,Q96P31;Fc receptor-like protein 3,gbmi-a-00001-nfe-b,Asthma,European,5,0.029067,...,3.050000e-33,0.024324,0.033809,generalized IVW,DF,1,1,0,1.0,SeqId_4440_15
7,ENSG00000164307,ERAP1,5,ARTS1,Q9NZ08;Endoplasmic reticulum aminopeptidase 1,gbmi-a-00001-nfe-b,Asthma,European,12,0.013050,...,6.410000e-09,0.008645,0.017456,generalized IVW,DF+HF,1,0,0,0.0,SeqId_4964_67
8,ENSG00000115594,IL1R1,2,IL-1 sRI,P14778;Interleukin-1 receptor type 1,gbmi-a-00001-nfe-b,Asthma,European,3,0.155661,...,3.430000e-17,0.119474,0.191848,generalized IVW,DF+HF,1,0,0,0.0,SeqId_2991_9
9,ENSG00000213676,ATF6B,6,ATF6B,Q99941;Cyclic AMP-dependent transcription fact...,gbmi-a-00001-nfe-b,Asthma,European,3,0.180044,...,3.430000e-24,0.145266,0.214821,generalized IVW,DF,1,0,0,1.0,SeqId_11387_3
